## This notebook contains a simple introduction to TF
### as used in:

* <font size=4 color='green'>MSTC seminar on Deep Learning & Tensorflow</font>

In [1]:
! pip show tensorflow

Name: tensorflow
Version: 0.10.0
Summary: TensorFlow helps the tensors flow
Home-page: http://tensorflow.org/
Author: Google Inc.
Author-email: opensource@google.com
License: Apache 2.0
Location: /gpfs/global_fs01/sym_shared/YPProdSpark/user/s35d-c92fe439b3aa21-1525a0f645bc/.local/lib/python2.7/site-packages
Requires: mock, numpy, six, protobuf, wheel


In [2]:
#! pip install --upgrade tensorflow

In [3]:
import tensorflow as tf

x=tf.constant(1.0)
W=tf.constant(6.0)
b=tf.constant(1.5)

y=x*W+b

print(y)

Tensor("add:0", shape=(), dtype=float32)


In [4]:
with tf.Session() as sess:
    print(sess.run(y))

7.5


### Now let's see a simple use of Variables in Tensorflow

In [5]:
x=tf.constant(1.0)
W=tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b=tf.Variable(tf.zeros([1]))

# Before starting, initialize the variables
# new versions of tf use: 
#init = tf.global_variables_initializer()
init = tf.initialize_all_variables()

y=x*W+b

with tf.Session() as sess:
    sess.run(init)
    for step in range(4):
        print(sess.run(y))



[-0.41041374]
[-0.41041374]
[-0.41041374]
[-0.41041374]


### Placeholders: computational graph entry points

In [10]:
x=tf.placeholder(tf.float32)
W=tf.Variable(tf.random_uniform([1], -1.0, 1.0))
b=tf.Variable(tf.zeros([1]))

# Before starting, initialize the variables
# new versions of tf use: 
#init = tf.global_variables_initializer()
init = tf.initialize_all_variables()

y=x*W+b

with tf.Session() as sess:
    sess.run(init)
    for step in range(4):
        print(sess.run(y,feed_dict={x:7.0}))


[-6.72060299]
[-6.72060299]
[-6.72060299]
[-6.72060299]
